In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

# Load your data (replace with your actual data loading)
# Assuming your data is in a CSV file named 'data.csv'
data = pd.read_csv('azureml://subscriptions/512a781e-d15a-4734-adca-96ec827531cb/resourcegroups/xwang3306-rg/workspaces/DVA_PROJECT/datastores/workspaceblobstore/paths/UI/2024-10-16_040004_UTC/consolidated_cleaned_neiss_2014_2023.csv')

[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
list(data.columns)

['CPSC_Case_Number',
 'Treatment_Date',
 'Age',
 'Sex',
 'Race',
 'Other_Race',
 'Hispanic',
 'Body_Part',
 'Diagnosis',
 'Other_Diagnosis',
 'Body_Part_2',
 'Diagnosis_2',
 'Other_Diagnosis_2',
 'Disposition',
 'Location',
 'Fire_Involvement',
 'Product_1',
 'Product_2',
 'Product_3',
 'Alcohol',
 'Drug',
 'Narrative',
 'Stratum',
 'PSU',
 'Weight',
 'Year',
 'Month',
 'Day']

In [7]:
import numpy as np
data['Disposition_recode']=np.nan
data.loc[((data['Disposition']==6) | (data['Disposition']==1)), 'Disposition_recode'] = 0
data.loc[((data['Disposition']==2)), 'Disposition_recode'] = 1
data.loc[((data['Disposition']==4)), 'Disposition_recode'] = 2
data.loc[((data['Disposition']==5)), 'Disposition_recode'] = 3
data.loc[((data['Disposition']==8)), 'Disposition_recode'] = 4
data=data[data['Disposition_recode'].notna()]

data['Disposition_recode_2']=0
data.loc[((data['Disposition_recode']>0)), 'Disposition_recode_2'] = 1

In [8]:
data['Disposition_recode'].value_counts()
data['Disposition_recode_2'].value_counts()

0    3142030
1     378362
Name: Disposition_recode_2, dtype: int64

In [17]:
def remove_after_dx(narrative):
  if isinstance(narrative, str):
    parts = narrative.split("DX", 1)
    if len(parts) > 1:
      return parts[0]
    else:
      return narrative  # No "DX:" found, return the original string
  else:
    return narrative  # Not a string, return as is


replace_list=['ANKLE', 'ARM', 'BODY_PART', 'CHEST', 'CONTUSION', 'CUT', 'EAR', 'ELBOW', 'EYE', 'FACE', 'FINGER', 'FOOT', 'FOREHEAD', 'FRACTURE', 'FX', 'HAND', 'HEAD', 'HIP', 'KNEE', 'LAC', 'LACERATION', 'LEG', 'LOC', 'LOSE', 'NECK', 'PAIN', 'SHOULDER', 
'SPRAIN', 'STRAIN', 'SWELL', 'THUMB', 'TOE', 'WRIST','ABRASION', 'ACHE', 'BREAK', 'BURN', 'CHIN', 'CUT', 'ER', 'FALL', 'FRACTURE', 'FX', 'HIT', 'INJURY', 'LACERATION', 'LIP', 'LOSE', 'LOC', 'MOUTH', 'NOSE', 'PAIN', 'RIB', 'SCALP', 'SPRAIN', 'STRAIN', 'SWELL', 'TOE', 'TWIST', 'WRIST']

for i in replace_list:
  data['Narrative'] = data['Narrative'].str.replace(i, '')

data['Narrative'] = data['Narrative'].apply(remove_after_dx)
data['Narrative'] = data['Narrative'].str.replace('YOM', '')
data['Narrative'] = data['Narrative'].str.replace('YOF', '')
data['Narrative'] = data['Narrative'].str.replace('YR', '')
data['Narrative'] = data['Narrative'].str.replace('OLD', '')
data['Narrative'] = data['Narrative'].str.replace('MALE', '')
data['Narrative'] = data['Narrative'].str.replace('FEMALE', '')
data['Narrative'] = data['Narrative'].str.replace(' YO ', '')
data['Narrative'] = data['Narrative'].str.replace('ACCIDENTALLY','')
data['Narrative'] = data['Narrative'].str.replace('AGO', '')
data['Narrative'] = data['Narrative'].str.replace('TODAY', '')
data['Narrative'] = data['Narrative'].str.replace('YESTERDAY', '')
data['Narrative'] = data['Narrative'].str.replace('PATIENT', '')
data['Narrative'] = data['Narrative'].str.replace(' PT ', '')
data['Narrative'] = data['Narrative'].str.replace('INJURY', '')
data['Narrative'] = data['Narrative'].str.replace('REPORT', '')
data['Narrative'] = data['Narrative'].str.replace('HURT', '')
data['Narrative'] = data['Narrative'].str.replace('INJ', '')
data['Narrative'] = data['Narrative'].str.replace('FELL', 'FALL')
data['Narrative'] = data['Narrative'].str.replace('INJURE', '')
data['Narrative'] = data['Narrative'].str.replace('JURED', '')
data['Narrative'] = data['Narrative'].str.replace('URED', '')
data['Narrative'] = data['Narrative'].str.replace(' ED', '')
data['Narrative'] = data['Narrative'].str.replace(' RT ', '')
data['Narrative'] = data['Narrative'].str.replace(' LT ', '')


In [18]:
core_col=['Disposition_recode_2','Age', 'Sex', 'Race','Location','Hispanic', 'Body_Part','Product_1' ,'Alcohol', 'Drug','Narrative']
data_core=data[core_col]
data_core_sample=data_core.sample(frac=0.1).reset_index(drop=True)

In [19]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')


corpus = data_core_sample['Narrative'].fillna('')

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stopwords.words('english')]
    tagged_tokens = nltk.pos_tag(tokens)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token, pos='v') for token, pos in tagged_tokens]
    return ' '.join(lemmatized_tokens)

data_core_sample['Processed_Narrative'] = corpus.apply(preprocess_text)

# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer(max_features=140, stop_words='english')

# Fit and transform the processed text
tfidf_matrix = vectorizer.fit_transform(data_core_sample['Processed_Narrative'])

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Concatenate the TF-IDF features with your existing data
data_ready = pd.concat([data_core_sample, tfidf_df], axis=1)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [20]:
list(data_ready.columns)

['Disposition_recode_2',
 'Age',
 'Sex',
 'Race',
 'Location',
 'Hispanic',
 'Body_Part',
 'Product_1',
 'Alcohol',
 'Drug',
 'Narrative',
 'Processed_Narrative',
 'aft',
 'anoth',
 'ation',
 'backwards',
 'balance',
 'ball',
 'bar',
 'baseball',
 'basketball',
 'bathroom',
 'bed',
 'bend',
 'bicycle',
 'bike',
 'body',
 'bottle',
 'box',
 'break',
 'broth',
 'car',
 'carpet',
 'carry',
 'catch',
 'cause',
 'cd',
 'chair',
 'class',
 'clean',
 'climb',
 'come',
 'concrete',
 'corn',
 'couch',
 'day',
 'days',
 'develop',
 'dog',
 'door',
 'dress',
 'drink',
 'drop',
 'excise',
 'facial',
 'fe',
 'feet',
 'felt',
 'fence',
 'flight',
 'floor',
 'fore',
 'forward',
 'game',
 'glass',
 'grind',
 'gym',
 'hd',
 'heavy',
 'helmet',
 'home',
 'hot',
 'house',
 'ing',
 'jump',
 'kick',
 'kitchen',
 'knife',
 'ladd',
 'land',
 'leave',
 'lift',
 'lose',
 'low',
 'metal',
 'miss',
 'mom',
 'moth',
 'nail',
 'night',
 'nurse',
 'open',
 'ov',
 'park',
 'pass',
 'piece',
 'play',
 'pool',
 'pop',

In [22]:
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_auc_score
from scipy.stats import mode

# Separate features (X) and target variable (y)
X = data_ready.drop(['Disposition_recode_2','Narrative','Processed_Narrative'], axis=1)  # ,'Processed_Narrative'] Replace 'target_variable' with your target column name
y = data_ready['Disposition_recode_2']

# Encode target variable if it's categorical
le = LabelEncoder()
y = le.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a scaler for numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(include=['number']))
X_test_scaled = scaler.transform(X_test.select_dtypes(include=['number']))

# Convert scaled features back to DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.select_dtypes(include=['number']).columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.select_dtypes(include=['number']).columns)

# Combine scaled numerical features with categorical features
X_train_final = X_train_scaled_df.copy()
X_test_final = X_test_scaled_df.copy()


# Define models and their parameter grids for hyperparameter tuning
models = {
    'KNN': (KNeighborsClassifier(), {'knn__n_neighbors': [3, 5, 7]}),
    'Random_Forest': (RandomForestClassifier(), {'random_forest__n_estimators': [100, 200], 'random_forest__max_depth': [4, 8]}),
    'XGBoost': (XGBClassifier(objective='binary:logistic'), {'xgboost__learning_rate': [0.1, 0.01], 'xgboost__max_depth': [3, 5]})
}

# Initialize list to store predictions from each model
ensemble_predictions = []
ensemble_probabilities = []

for model_name, (model, param_grid) in models.items():
    print(f"Training {model_name}...")

    # Create a pipeline for preprocessing and model training
    pipeline = Pipeline([
        (model_name.lower(), model)
    ])

    # Perform hyperparameter tuning using GridSearchCV with AUC score
    grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='roc_auc')
    grid_search.fit(X_train_final, y_train)

    # Get the best model from the grid search
    best_model = grid_search.best_estimator_

    # Make predictions and predict probabilities on the test set
    y_pred = best_model.predict(X_test_final)
    y_prob = best_model.predict_proba(X_test_final)[:, 1]  # Probability for the positive class
    ensemble_predictions.append(y_pred)
    ensemble_probabilities.append(y_prob)

    # Evaluate the model using accuracy, F1-score, and AUC
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    print(f"{model_name} Accuracy: {accuracy}")
    print(f"{model_name} F1 Score: {f1}")
    print(f"{model_name} AUC: {auc}")
    print(classification_report(y_test, y_pred))

# Implement ensemble method (majority voting for predictions, average for probabilities)
ensemble_predictions = np.array(ensemble_predictions)
ensemble_pred_final = mode(ensemble_predictions, axis=0)[0].flatten()

# Average the probabilities for the ensemble AUC
ensemble_probabilities = np.mean(ensemble_probabilities, axis=0)

# Evaluate the ensemble model using accuracy, F1-score, and AUC
ensemble_accuracy = accuracy_score(y_test, ensemble_pred_final)
ensemble_f1 = f1_score(y_test, ensemble_pred_final)
ensemble_auc = roc_auc_score(y_test, ensemble_probabilities)
print("Ensemble Accuracy:", ensemble_accuracy)
print("Ensemble F1 Score:", ensemble_f1)
print("Ensemble AUC:", ensemble_auc)
print(classification_report(y_test, ensemble_pred_final))

Training KNN...
KNN Accuracy: 0.8862771275991365
KNN F1 Score: 0.2047869699076373
KNN AUC: 0.6948368117014451
              precision    recall  f1-score   support

           0       0.90      0.98      0.94     62792
           1       0.42      0.14      0.20      7616

    accuracy                           0.89     70408
   macro avg       0.66      0.56      0.57     70408
weighted avg       0.85      0.89      0.86     70408

Training Random_Forest...
Random_Forest Accuracy: 0.8918304738097943
Random_Forest F1 Score: 0.0
Random_Forest AUC: 0.7794048484890357
              precision    recall  f1-score   support

           0       0.89      1.00      0.94     62792
           1       0.00      0.00      0.00      7616

    accuracy                           0.89     70408
   macro avg       0.45      0.50      0.47     70408
weighted avg       0.80      0.89      0.84     70408

Training XGBoost...
[15:52:52] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the defaul

/anaconda/envs/azureml_py38/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/anaconda/envs/azureml_py38/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/anaconda/envs/azureml_py38/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

In [8]:
xxx

NameError: name 'xxx' is not defined